In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torchvision import models
import cv2
from utils.utilities import *

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image,preprocess_image
from pytorch_grad_cam.utils.model_targets import RawScoresOutputTarget

In [ ]:
groups = get_groups()

In [ ]:
final_data = groups.iloc[7:]
final_data

# Calculating similarity

In [ ]:
class ResnetFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super(ResnetFeatureExtractor, self).__init__()
        self.model = model
        self.feature_extractor = torch.nn.Sequential(*list(self.model.children())[:-1])
                
    def __call__(self, x):
        return self.feature_extractor(x)[:, :, 0, 0]
    
def load_img(path,resize_shape=None) -> np.ndarray:
    img = cv2.imread(path)
    if resize_shape == None:
        return np.float32(img)/255
    return np.float32(cv2.resize(img,resize_shape))/255
    
def img_to_tensor(img,device):
    return preprocess_image(
        img,
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ).to(device)

def calculate_similarity(inspiration, final_image, model:ResnetFeatureExtractor, device):
    inspiration_tensor = img_to_tensor(inspiration,device)
    final_image_tensor = img_to_tensor(final_image,device)

    features_inspiration = model(inspiration_tensor)[0,:]
    features_final = model(final_image_tensor)[0,:]

    return np.float32(torch.nn.functional.cosine_similarity(features_inspiration,features_final,dim=0).cpu().detach().numpy())

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(pretrained=True).to(device)
resnet.eval()
model = ResnetFeatureExtractor(resnet)

In [ ]:
for i in range(len(final_data)):
    print(f'Calculating cross-similarity for group {final_data.iloc[i].group_code}')
    ai = final_data.iloc[i].ai_images
    web = final_data.iloc[i].web_images
    final = final_data.iloc[i].final_submissions
    ai_aggregated_similarity, web_aggregated_similarity = 0, 0
    max_similarity, picture1, picture2 = 0, "", ""

    for ai_photo in ai:
        im1 = load_img(ai_photo)
        for final_photo in final:
            im2 = load_img(final_photo)
            similarity = calculate_similarity(im1, im2, model, device) # placebo for now
            ai_aggregated_similarity += similarity
            if similarity > max_similarity:
                max_similarity = similarity
                picture1, picture2 = im1, im2
            
    print(f'AI total similarity:\t{ai_aggregated_similarity}')
    
    for web_photo in web:
        im1 = load_img(web_photo)
        for final_photo in final:
            im2 = load_img(final_photo)
            similarity = calculate_similarity(im1, im2, model, device) # placebo for now
            web_aggregated_similarity += similarity
            if similarity > max_similarity:
                max_similarity = similarity
                picture1, picture2 = im1, im2
    
    print(f'Web total similarity:\t{web_aggregated_similarity}')

    try:
        print('Two most similar images are these:')
        plt.subplot(1, 2, 1)
        plt.imshow(picture1)
        plt.title("Inspiration")
        plt.subplot(1, 2, 2)
        plt.imshow(picture2)
        plt.title("Submission")
        plt.show()
    except Exception:
        print('No images to compare')

Very primitive approach. Future ideas:
 - Use selective search or region proposal network across all images at the same time to extract the features from inspiration and final images
 - Create some ranking of highest similarity between images and display the most similar images.